In [1]:
import sys
import os

sys.path.append(os.path.abspath("../.."))


from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END

# Create a simple example graph
simple_graph = Graph()


# Define some example actions
@simple_graph.node()
def escape():
    print("Starting workflow")


@simple_graph.node()
def process_data():
    print("Processing data")


@simple_graph.node()
def validate():
    print("Validating results")


@simple_graph.node()
def aa():
    print("Validating results")


@simple_graph.node()
def bb():
    print("Validating results")


@simple_graph.node()
def dd():
    print("Validating results")


@simple_graph.node()
def cc():
    print("Validating results")


@simple_graph.node()
def hh():
    print("Validating results")


@simple_graph.node()
def prep():
    print("Workflow complete")


# Add edges to create workflow
simple_graph.add_edge(START, "process_data")
simple_graph.add_edge("process_data", "validate")
simple_graph.add_edge("validate", "escape")
simple_graph.add_edge("escape", "dd")
simple_graph.add_edge("escape", "cc")
simple_graph.add_edge("cc", "hh")
simple_graph.add_edge("dd", "hh")
simple_graph.add_edge("hh", "prep")
simple_graph.add_edge("validate", "aa")
simple_graph.add_edge("aa", "bb")
simple_graph.add_edge("bb", "prep")
simple_graph.add_edge("prep", END)

# Visualize the simple_graph
simple_graph.compile()

## Testing execution plan conversion


In [2]:
from rich import print as rprint

rprint(simple_graph._convert_execution_plan())

[
    ExecutableNode(
        node_name='process_data',
        task_list=[<function process_data at 0x105e863e0>],
        execution_type='sequential'
    ),
    ExecutableNode(
        node_name='validate',
        task_list=[<function validate at 0x105e86480>],
        execution_type='sequential'
    ),
    ExecutableNode(
        node_name="group_['aa', 'bb']",
        task_list=[
            ExecutableNode(
                node_name='group_aa',
                task_list=[
                    ExecutableNode(
                        node_name='aa',
                        task_list=[<function aa at 0x105e86520>],
                        execution_type='sequential'
                    ),
                    ExecutableNode(
                        node_name='bb',
                        task_list=[<function bb at 0x105e865c0>],
                        execution_type='sequential'
                    )
                ],
                execution_type='parallel'
            ),
            ExecutableNode(
                node_name='group_escape',
                task_list=[
                    ExecutableNode(
                        node_name='escape',
                        task_list=[<function escape at 0x105937ce0>],
                        execution_type='sequential'
                    ),
                    ExecutableNode(
                        node_name='group_cc',
                        task_list=[
                            ExecutableNode(
                                node_name='cc',
                                task_list=[<function cc at 0x105e86700>],
                                execution_type='sequential'
                            ),
                            ExecutableNode(
                                node_name='dd',
                                task_list=[<function dd at 0x105e86660>],
                                execution_type='sequential'
                            )
                        ],
                        execution_type='parallel'
                    ),
                    ExecutableNode(
                        node_name='hh',
                        task_list=[<function hh at 0x105e867a0>],
                        execution_type='sequential'
                    )
                ],
                execution_type='parallel'
            )
        ],
        execution_type='parallel'
    ),
    ExecutableNode(node_name='prep', task_list=[<function prep at 0x105e86840>], execution_type='sequential')
]

In [7]:
def extract_executable_node(executable_node):
    if len(executable_node.task_list) <= 1:
        return ( executable_node.execution_type)
    else:
        return [extract_executable_node(task) for task in executable_node.task_list]

def check_node_count(executable_nodes_info_item):
    if isinstance(executable_nodes_info_item, list):
        return len(executable_nodes_info_item)
    else:
        return 1

rprint([extract_executable_node(task) for task in simple_graph._convert_execution_plan()])

[check_node_count(item) for item]

[
    (<function process_data at 0x105e863e0>, 'sequential'),
    (<function validate at 0x105e86480>, 'sequential'),
    [
        [(<function aa at 0x105e86520>, 'sequential'), (<function bb at 0x105e865c0>, 'sequential')],
        [
            (<function escape at 0x105937ce0>, 'sequential'),
            [(<function cc at 0x105e86700>, 'sequential'), (<function dd at 0x105e86660>, 'sequential')],
            (<function hh at 0x105e867a0>, 'sequential')
        ]
    ],
    (<function prep at 0x105e86840>, 'sequential')
]

[1, 1, 2, 1]

In [ ]:
rprint(simple_graph.execution_plan)

## Testing execution plan conversion


In [ ]:
rprint(simple_graph.edges)